In [1]:
import dill as pickle
from collections import Counter, defaultdict
import csv
import re
import numpy as np
import os
from evlex_shared_scripts import *

In [2]:
corpus_fn = './files/corpus_doreco.p'
files_metadata = './files/doreco_files_metadata.csv'

In [3]:
genres = {'personal narrative','traditional narrative','conversation','procedural'} # TODO: run with other genres
raw_corpus = pickle.load(open(corpus_fn, 'rb'))         
corpus = select_genres(raw_corpus, files_metadata, genres)

[('traditional narrative', 626), ('personal narrative', 436), ('stimulus retelling', 158), ('conversation', 108), ('procedural', 76), ('procedural/conversation', 2)]


# BERT tokenization

In [4]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = TFBertModel.from_pretrained('bert-base-cased')

2024-12-17 16:19:32.580208: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-17 16:19:32.586569: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-17 16:19:32.641798: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-17 16:19:32.680106: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734470372.722092  177733 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734470372.73

In [5]:
indices = defaultdict(lambda : defaultdict(lambda : []))
input_ids = defaultdict(lambda : {})
for l in corpus:
    i = 0
    for f in corpus[l]:
        for ix,(li,elt) in enumerate(sorted(corpus[l][f].items())):
            ft = elt['ft']
            input_id = tf.constant(tokenizer.encode(ft))[None, :]
            if i == 0 :
                print(l, ix, ft, '\n', [tokenizer.decode([iii]) for iii in input_id[:,(0 if ix == 0 else 1):].numpy()[0]],'\n',
                 tokenizer.decode([iii for iii in input_id[:,(0 if ix == 0 else 1):].numpy()[0]]))
            if ix > 0: 
                input_id = input_id[:,1:]
            #
            if ix == 0: 
                input_ids[l][f] = input_id
            else: 
                input_ids[l][f] = tf.concat((input_ids[l][f], input_id), 1)
            indices[l][f].extend([li] * input_id.shape[1])
            i += 1
        #print(f, input_ids[l][f].shape, len(indices[l]))

anal1239.csv 0 Anal proverbs 
 ['[CLS]', 'Ana', '##l', 'prove', '##rb', '##s', '[SEP]'] 
 [CLS] Anal proverbs [SEP]
apah1238.csv 0 hello mama 
 ['[CLS]', 'hello', 'ma', '##ma', '[SEP]'] 
 [CLS] hello mama [SEP]
arap1274.csv 0 Trickster was walking along. 
 ['[CLS]', 'Trick', '##ster', 'was', 'walking', 'along', '.', '[SEP]'] 
 [CLS] Trickster was walking along. [SEP]
bain1259.csv 0 The glass that has been put on the table 
 ['[CLS]', 'The', 'glass', 'that', 'has', 'been', 'put', 'on', 'the', 'table', '[SEP]'] 
 [CLS] The glass that has been put on the table [SEP]
beja1238.csv 0 These men, first, 
 ['[CLS]', 'These', 'men', ',', 'first', ',', '[SEP]'] 
 [CLS] These men, first, [SEP]
bora1263.csv 0 Let's see, tell him how you have fallen into the water. 
 ['[CLS]', 'Let', "'", 's', 'see', ',', 'tell', 'him', 'how', 'you', 'have', 'fallen', 'into', 'the', 'water', '.', '[SEP]'] 
 [CLS] Let's see, tell him how you have fallen into the water. [SEP]
cabe1245.csv 0 My ancestors were these Cab

In [6]:
def get_slices(indices, padding = 100, max_tok = 512):
    start = 0
    end = len(indices[0])
    s_outer, e_outer = 0, min(max_tok, end)
    s_inner, e_inner = s_outer, e_outer - (padding if e_outer != end else 0)
    slices = []
    slices.append(((s_outer, e_outer),(s_inner - s_outer, (e_outer-s_outer) - (e_outer-e_inner))))
    while s_outer != end and e_inner != end:
        s_outer = min(end, s_outer + (max_tok - 2 * padding))
        e_outer = min(end, e_outer + (max_tok - 2 * padding))
        s_inner = s_outer + padding
        e_inner = e_outer - (padding if e_outer != end else 0)
        if e_inner == end:
            s_outer = max(0, e_outer-max_tok)
        
        slices.append(((s_outer, e_outer),(s_inner - s_outer, (e_outer-s_outer) - (e_outer-e_inner))))
    #print((s_outer, e_outer),(s_inner,e_inner))
    return slices

In [9]:
dn = './bert_vectors/'
for l in input_ids:
    print(l.upper())
    if not os.path.isdir(dn + l[:8]):
        os.mkdir(dn + l[:8])
    for f in input_ids[l]:
        slices = get_slices(input_ids[l][f])
        for (so,eo),(si,ei) in slices:
            input_id = input_ids[l][f][:,so:eo]
            outputs = model(input_id)
            if si == 0:
                bert = outputs[0][:,si:ei,:]
            else:
                bert = tf.concat((bert, outputs[0][:,si:ei,:]), 1)
    
        print('\t',f,input_ids[l][f].shape,bert.shape)
        np.save('%s/%s/%s.npy' % (dn,l[:8],f), bert.numpy())

ANAL1239.CSV
	 doreco_anal1239_anm_20151216_10_Ngahring_WB_proverbs1 (1, 2872) (1, 2872, 768)
	 doreco_anal1239_anm_20151216_10_Ngahring_WB_riddles (1, 782) (1, 782, 768)
	 doreco_anal1239_anm_20152111_Ngahring_folkfore_60 (1, 864) (1, 864, 768)
	 doreco_anal1239_anm_20152111_Ngahring_PO_55 (1, 1145) (1, 1145, 768)
	 doreco_anal1239_anm_20152111_Ngahring_PO_56 (1, 1252) (1, 1252, 768)
	 doreco_anal1239_anm_20152111_Ngahring_PO_61 (1, 658) (1, 658, 768)
	 doreco_anal1239_anm_20152111_Ngahring_WB_dead_woman_and_her_children (1, 661) (1, 661, 768)
	 doreco_anal1239_anm_20152111_Ngahring_WB_frog_and_jackal (1, 277) (1, 277, 768)
	 doreco_anal1239_anm_20160220_Mohtung_PO_15 (1, 173) (1, 173, 768)
	 doreco_anal1239_anm_20160220_Mohtung_PO_3 (1, 435) (1, 435, 768)
	 doreco_anal1239_anm_20160220_Mohtung_PO_8 (1, 194) (1, 194, 768)
	 doreco_anal1239_anm_20160220_Ngahring_Folkflore_53 (1, 1501) (1, 1501, 768)
	 doreco_anal1239_anm_20160220_Ngahring_PO_52 (1, 1342) (1, 1342, 768)
	 doreco_anal123

	 doreco_urum1249_UUM-TXT-MR-00000-A04 (1, 121) (1, 121, 768)
	 doreco_urum1249_UUM-TXT-MR-00000-A05 (1, 193) (1, 193, 768)
	 doreco_urum1249_UUM-TXT-MR-00000-A06 (1, 159) (1, 159, 768)
	 doreco_urum1249_UUM-TXT-MR-00000-A07 (1, 244) (1, 244, 768)
	 doreco_urum1249_UUM-TXT-MR-00000-A08 (1, 168) (1, 168, 768)
	 doreco_urum1249_UUM-TXT-MR-00000-A09 (1, 328) (1, 328, 768)
	 doreco_urum1249_UUM-TXT-MR-00000-A10 (1, 159) (1, 159, 768)
	 doreco_urum1249_UUM-TXT-MR-00000-A11 (1, 234) (1, 234, 768)
	 doreco_urum1249_UUM-TXT-MR-00000-A12 (1, 254) (1, 254, 768)
	 doreco_urum1249_UUM-TXT-MR-00000-A13 (1, 198) (1, 198, 768)
	 doreco_urum1249_UUM-TXT-MR-00000-A14 (1, 169) (1, 169, 768)
	 doreco_urum1249_UUM-TXT-MR-00000-A15 (1, 150) (1, 150, 768)
	 doreco_urum1249_UUM-TXT-MR-00000-A16 (1, 345) (1, 345, 768)
	 doreco_urum1249_UUM-TXT-MR-00000-B01 (1, 152) (1, 152, 768)
	 doreco_urum1249_UUM-TXT-MR-00000-B02 (1, 116) (1, 116, 768)
	 doreco_urum1249_UUM-TXT-MR-00000-B03 (1, 134) (1, 134, 768)
	 doreco

In [12]:
pickle.dump(indices, open('./files/indices.p', 'wb'))
pickle.dump(input_ids, open('./files/input_ids.p', 'wb'))